In [1]:
pip install cbsodata

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import cbsodata 

# Downloading table list
toc = pd.DataFrame(cbsodata.get_table_list())

# Downloading entire dataset (can take up to 30s)
# data = pd.DataFrame(cbsodata.get_data('83765NED'))
# print(data.head())

# Downloading metadata
# metadata = pd.DataFrame(cbsodata.get_meta('83765NED', 'DataProperties'))
# print(metadata[['Key','Title']])

In [3]:
# Comment out: Use CTRL+/

# Import functions from libraries.
from collections import Counter

from pandas.api.types import is_numeric_dtype

from sklearn.compose import ColumnTransformer

from sklearn.decomposition import PCA

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn import metrics

from scipy.stats import pearsonr

# Import whole libraries. '%matplotlib inline' saves matplotlib graphs with notebook.
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sys
import math

# Set default figure size. This replaces 'plt.figure(figsize=(20,10))' in every cell.
plt.rcParams['figure.figsize'] = (20, 10)

# How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option('display.max_rows', 20) 

In [4]:
# To concatenate values.
def f_concat(l_input):

    # Initialize.
    dummy = ""
    n_len = len(l_input)

    if n_len == 0:
        return None
    
    if n_len == 1:
        return l_input[0]

    # Loop through text elements.
    for i in range(n_len-1):
        dummy = dummy + l_input[i] + ", "

    # Append last element.
    dummy = dummy + "and " + l_input[n_len-1]

    # Return result.
    return dummy

In [5]:
## DEFINE FUNCTIONS
# To describe the dataframe
def f_describe(df_input, n_top = 10):

    # Determine columns of the same data type.
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html
    # https://numpy.org/doc/stable/reference/arrays.scalars.html
    df_integer  = df_input.select_dtypes(include = np.integer)
    df_floating = df_input.select_dtypes(include = np.floating)
    df_object   = df_input.select_dtypes(include = object)
    df_other    = df_input.select_dtypes(exclude = [np.integer, np.floating, object])

    # Overall stats
    print("The data:\n")
    print(f"-> Dimension:        {df_input.shape[0]} observations and {df_input.shape[1]} columns.\n")
    print(f"-> Size:             {round(sys.getsizeof(df_input)/1024/1024, 1)} MB.\n")
    print(f"-> Integer data in:  {f_concat(np.sort(df_integer.columns))}.\n")
    print(f"-> Floating data in: {f_concat(np.sort(df_floating.columns))}.\n")
    print(f"-> Object data in:   {f_concat(np.sort(df_object.columns))}.\n\n")
    print(f"-> Other data in:   {f_concat(np.sort(df_other.columns))}.\n\n")

    # Show first 'n_top' rows of the data.
    print("Show data (first " + str(n_top) + " rows):\n")
    display(df_input.head(n_top))
  
    # Describe integer columns
    if len(df_integer.columns):
        print("\n\nDescribe integer data:")
        display(df_integer.describe())

    # Describe floating columns
    if len(df_floating.columns):
        print("\n\nDescribe floating data:")
        display(df_floating.describe())

    # Describe object columns
    if len(df_object.columns):
        print("\n\nDescribe object data:")
        display(df_object.describe())

    # Describe other columns
    if len(df_other.columns):
        print("\n\nDescribe other data:")
        display(df_other.describe())

    # Show columns with missing data.
    v_total_missing   = df_input.isnull().sum()
    v_percent_missing = round(v_total_missing / df_input.shape[0] * 100, 1)
    v_missing_type    = df_input.dtypes

    df_missing_data = pd.DataFrame({'type': v_missing_type, 'total': v_total_missing, 'percent': v_percent_missing})
    df_missing_data = df_missing_data.sort_values(by='total', ascending = False)
    df_missing_data = df_missing_data[df_missing_data.total > 0]

    if(df_missing_data.shape[0] == 0):
      print("")
      print("None of the columns have missing data!")
    else:
      print("\n\nShow missing data:")
      display(df_missing_data)



In [6]:
# To give information on a column in a dataframe.
def f_info(df_input, c_col, n_top = 10, n_font_size = 20):

    # Do not calculate the frequency table in case the feature has unique values.
    #if (len(set(df_input[c_col])) == len(df_input[c_col])):
    if (df_input[c_col]).is_unique:

      print("Column '" + c_col + "' consists of unique values.\n")

    if (len(set(df_input[c_col])) == 1):
      print("Column '" + c_col + "' consists of the same value.\n")

    # Bereken frequenties.
    c = Counter(df_input[c_col])

    # Converteer naar data frame.
    df_output         = pd.DataFrame(list(c.items()))

    # Hernoem kolomnamen.
    df_output.columns = ["level", "n"]

    # Bereken percentage.
    df_output["perc"] = round(100 * df_output["n"] / df_input.shape[0], 1).astype(str) + "%"

    # Sorteer data frame op frequentie.
    df_output         = df_output.sort_values(by = "n", ascending = False)

    if(df_output.shape[0] <= n_top):
            c_message = "we show all " + str(df_output.shape[0]) + " levels:"
            n_top     = df_output.shape[0]
            
    else:
            c_message = "we show the Top-" + str(n_top) + " of the " + str(df_output.shape[0]) + " levels:"
        
    # Print header
    print("Frequency of values in colum '" + c_col + "', " + c_message + "\n")

    #print(f"Number of NA: {df_input[c_col].isna().sum()} ({round(100 * df_input[c_col].isna().sum() / df_input.shape[0], 1)}%)\n")
            
    display(df_output.head(n_top))

    print("\n")

    # Plot frequency n_top elements.
    ax = df_input[c_col].value_counts(sort = True, ascending = False)[0:n_top].plot(kind='barh')
    ax.invert_yaxis()
    ax.set_ylabel(c_col)

# https://stackoverflow.com/questions/3899980/how-to-change-the-font-size-on-a-matplotlib-plot
    for item in (ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(n_font_size)

    for item in [ax.xaxis.label, ax.yaxis.label]:
        item.set_fontsize(n_font_size + 4)

In [7]:
#This is the sickleave numbers on NL level. Pls check https://opendata.cbs.nl/statline/?ts=1588842622385#/CBS/nl/dataset/80072ned/table
df = pd.DataFrame(cbsodata.get_data('80072NED'))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         5148 non-null   int64  
 1   BedrijfskenmerkenSBI2008   5148 non-null   object 
 2   Perioden                   5148 non-null   object 
 3   Ziekteverzuimpercentage_1  4838 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 161.0+ KB


In [9]:
f_describe(df)

The data:

-> Dimension:        5148 observations and 4 columns.

-> Size:             0.8 MB.

-> Integer data in:  ID.

-> Floating data in: Ziekteverzuimpercentage_1.

-> Object data in:   BedrijfskenmerkenSBI2008, and Perioden.


-> Other data in:   None.


Show data (first 10 rows):



,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7
4,4,A-U Alle economische activiteiten,1996,4.7
5,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9
6,6,A-U Alle economische activiteiten,1997 2e kwartaal,4.5
7,7,A-U Alle economische activiteiten,1997 3e kwartaal,4.1
8,8,A-U Alle economische activiteiten,1997 4e kwartaal,4.9
9,9,A-U Alle economische activiteiten,1997,4.6




Describe integer data:


,ID
count,5148.000000
mean,2573.500000
std,1486.243923
min,0.000000
25%,1286.750000
50%,2573.500000
75%,3860.250000
max,5147.000000




Describe floating data:


,Ziekteverzuimpercentage_1
count,4838.000000
mean,4.478648
std,1.258787
min,1.400000
25%,3.500000
50%,4.400000
75%,5.300000
max,10.100000




Describe object data:


,BedrijfskenmerkenSBI2008,Perioden
count,5148,5148
unique,39,132
top,S Overige dienstverlening,2016 2e kwartaal
freq,132,39




Show missing data:


,type,total,percent
Ziekteverzuimpercentage_1,float64,310,6.0


In [10]:
#We create now a slice of the dataframe only containing sickleave on national level.
df_nl = df.loc[(df['BedrijfskenmerkenSBI2008'] == "A-U Alle economische activiteiten"),:]

In [11]:
df_nl

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7
4,4,A-U Alle economische activiteiten,1996,4.7
...,...,...,...,...
127,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6
128,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4
129,129,A-U Alle economische activiteiten,2021,4.9
130,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3


In [12]:
#We create now a dataframe only containing quarterly numbers
df_nl_q = df_nl.loc[(df_nl['Perioden'].str.contains('kwartaal')),:]

In [13]:
df_nl_q

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7
5,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9
...,...,...,...,...
126,126,A-U Alle economische activiteiten,2021 2e kwartaal,4.7
127,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6
128,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4
130,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3


In [14]:
DeepnoteChart(df_nl_q, """{"data":{"name":"placeholder"},"mark":{"type":"line","tooltip":true},"height":440,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"Perioden","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Ziekteverzuimpercentage_1","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"BedrijfskenmerkenSBI2008","scale":{"type":"linear","zero":false}}},"width":"container","config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

In [15]:
df_nlq = df_nl_q.copy()
conditions = [
    (df_nl_q['Perioden'].str.contains('1e')),
    (df_nl_q['Perioden'].str.contains('2e')),
    (df_nl_q['Perioden'].str.contains('3e')),
    (df_nl_q['Perioden'].str.contains('4e')) ]

season = ['Winter', 'Spring', 'Summer', 'Autumn']

df_nlq['Season'] = np.select(conditions, season)

In [16]:
df_nlq

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1,Season
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5,Winter
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6,Spring
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0,Summer
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7,Autumn
5,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9,Winter
...,...,...,...,...,...
126,126,A-U Alle economische activiteiten,2021 2e kwartaal,4.7,Spring
127,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6,Summer
128,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4,Autumn
130,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3,Winter


In [17]:
DeepnoteChart(df_nl_q, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":440,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}},"width":"container","config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

In [18]:
#Table arbeidsdeelname - https://www.cbs.nl/nl-nl/cijfers/detail/80590ned?q=80590ned  ; Arbeidsdeelname en werkloosheid per maand
ad = pd.DataFrame(cbsodata.get_data('80590NED'))

In [19]:
#Table vacatures
vacatures = pd.DataFrame(cbsodata.get_data('80474ned'))

In [20]:
ad

,ID,Geslacht,Leeftijd,Perioden,NietSeizoengecorrigeerd_1,Seizoengecorrigeerd_2,NietSeizoengecorrigeerd_3,Seizoengecorrigeerd_4,NietSeizoengecorrigeerd_5,Seizoengecorrigeerd_6,NietSeizoengecorrigeerd_7,Seizoengecorrigeerd_8,NietSeizoengecorrigeerd_9,Seizoengecorrigeerd_10,NietSeizoengecorrigeerd_11,Seizoengecorrigeerd_12,NietSeizoengecorrigeerd_13,Seizoengecorrigeerd_14
0,0,Totaal,15 tot 75 jaar,2003,8451.0,NaN,7951.0,NaN,500.0,NaN,5.9,NaN,3665.0,NaN,69.8,NaN,65.6,NaN
1,1,Totaal,15 tot 75 jaar,2004,8502.0,NaN,7926.0,NaN,576.0,NaN,6.8,NaN,3654.0,NaN,69.9,NaN,65.2,NaN
2,2,Totaal,15 tot 75 jaar,2005,8580.0,NaN,7982.0,NaN,597.0,NaN,7.0,NaN,3610.0,NaN,70.4,NaN,65.5,NaN
3,3,Totaal,15 tot 75 jaar,2006,8631.0,NaN,8106.0,NaN,525.0,NaN,6.1,NaN,3590.0,NaN,70.6,NaN,66.3,NaN
4,4,Totaal,15 tot 75 jaar,2007,8797.0,NaN,8331.0,NaN,466.0,NaN,5.3,NaN,3460.0,NaN,71.8,NaN,68.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,4015,Vrouwen,45 tot 75 jaar,2022 2e kwartaal,1925.0,1922.0,1873.0,1871.0,52.0,51.0,2.7,2.6,1444.0,1447.0,57.1,57.0,55.6,55.5
4016,4016,Vrouwen,45 tot 75 jaar,2022 juli,1919.0,1930.0,1872.0,1877.0,47.0,53.0,2.4,2.7,1451.0,1440.0,56.9,57.3,55.5,55.7
4017,4017,Vrouwen,45 tot 75 jaar,2022 augustus,1938.0,1935.0,1884.0,1877.0,54.0,57.0,2.8,3.0,1432.0,1435.0,57.5,57.4,55.9,55.7
4018,4018,Vrouwen,45 tot 75 jaar,2022 september,1942.0,1940.0,1882.0,1881.0,59.0,59.0,3.1,3.0,1429.0,1430.0,57.6,57.6,55.8,55.8


In [21]:
#Table arbeidsdeelname - Labor participation (niet seizoen gecorrigeerd)
ad_nc = pd.DataFrame(cbsodata.get_data('85264NED'))

In [22]:
ad_nc

,ID,Geslacht,Leeftijd,HoogstBehaaldOnderwijsniveau,Perioden,BeroepsEnNietBeroepsbevolking_1,Beroepsbevolking_2,WerkzameBeroepsbevolking_3,Werknemer_4,WerknemerMetVasteArbeidsrelatie_5,...,k_12Tot20UurPerWeek_15,k_20Tot35UurPerWeek_16,k_20Tot28UurPerWeek_17,k_28Tot35UurPerWeek_18,Voltijd_19,WerklozeBeroepsbevolking_20,Werkloosheidspercentage_21,NietBeroepsbevolking_22,BrutoArbeidsparticipatie_23,NettoArbeidsparticipatie_24
0,0,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 1e kwartaal,12617,9162,8427,7205,4782,...,592,2311,1224,1087,4465,735,8.0,3454,72.6,66.8
1,1,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 2e kwartaal,12631,9182,8436,7215,4753,...,599,2321,1210,1111,4464,747,8.1,3448,72.7,66.8
2,2,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 3e kwartaal,12645,9208,8443,7204,4706,...,592,2331,1215,1117,4491,764,8.3,3437,72.8,66.8
3,3,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 4e kwartaal,12659,9195,8426,7171,4651,...,587,2315,1184,1131,4444,768,8.4,3464,72.6,66.6
4,4,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013,12638,9187,8433,7199,4723,...,592,2320,1208,1111,4466,754,8.2,3451,72.7,66.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16210,16210,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021 3e kwartaal,1,0,0,0,0,...,0,0,0,0,0,0,NaN,1,NaN,NaN
16211,16211,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021 4e kwartaal,2,0,0,0,0,...,0,0,0,0,0,0,NaN,2,NaN,NaN
16212,16212,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021,2,0,0,0,0,...,0,0,0,0,0,0,NaN,2,NaN,NaN
16213,16213,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2022 1e kwartaal,4,0,0,0,0,...,0,0,0,0,0,0,NaN,4,NaN,NaN


In [23]:
#We create now a dataframe only containing quarterly numbers
ad_q = ad.loc[(ad['Perioden'].str.contains('kwartaal')),:]

In [24]:
ad_q

,ID,Geslacht,Leeftijd,Perioden,NietSeizoengecorrigeerd_1,Seizoengecorrigeerd_2,NietSeizoengecorrigeerd_3,Seizoengecorrigeerd_4,NietSeizoengecorrigeerd_5,Seizoengecorrigeerd_6,NietSeizoengecorrigeerd_7,Seizoengecorrigeerd_8,NietSeizoengecorrigeerd_9,Seizoengecorrigeerd_10,NietSeizoengecorrigeerd_11,Seizoengecorrigeerd_12,NietSeizoengecorrigeerd_13,Seizoengecorrigeerd_14
19,19,Totaal,15 tot 75 jaar,2003 1e kwartaal,8429.0,8441.0,7940.0,7985.0,489.0,456.0,5.8,5.4,3671.0,3659.0,69.7,69.8,65.6,66.0
23,23,Totaal,15 tot 75 jaar,2003 2e kwartaal,8440.0,8452.0,7949.0,7959.0,490.0,493.0,5.8,5.8,3671.0,3659.0,69.7,69.8,65.6,65.7
27,27,Totaal,15 tot 75 jaar,2003 3e kwartaal,8473.0,8464.0,7977.0,7951.0,496.0,513.0,5.8,6.1,3649.0,3658.0,69.9,69.8,65.8,65.6
31,31,Totaal,15 tot 75 jaar,2003 4e kwartaal,8464.0,8466.0,7938.0,7927.0,526.0,538.0,6.2,6.4,3668.0,3666.0,69.8,69.8,65.4,65.3
35,35,Totaal,15 tot 75 jaar,2004 1e kwartaal,8475.0,8479.0,7885.0,7919.0,590.0,559.0,7.0,6.6,3667.0,3663.0,69.8,69.8,64.9,65.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,4002,Vrouwen,45 tot 75 jaar,2021 3e kwartaal,1895.0,1897.0,1842.0,1842.0,52.0,55.0,2.8,2.9,1482.0,1479.0,56.1,56.2,54.6,54.5
4006,4006,Vrouwen,45 tot 75 jaar,2021 4e kwartaal,1902.0,1905.0,1850.0,1853.0,52.0,52.0,2.7,2.7,1470.0,1468.0,56.4,56.5,54.9,54.9
4011,4011,Vrouwen,45 tot 75 jaar,2022 1e kwartaal,1912.0,1911.0,1859.0,1862.0,53.0,49.0,2.8,2.6,1455.0,1456.0,56.8,56.8,55.2,55.3
4015,4015,Vrouwen,45 tot 75 jaar,2022 2e kwartaal,1925.0,1922.0,1873.0,1871.0,52.0,51.0,2.7,2.6,1444.0,1447.0,57.1,57.0,55.6,55.5


In [25]:
#We create now a dataframe only containing quarterly numbers
adnc_q = ad_nc.loc[(ad_nc['Perioden'].str.contains('kwartaal')),:]

In [26]:
adnc_q

,ID,Geslacht,Leeftijd,HoogstBehaaldOnderwijsniveau,Perioden,BeroepsEnNietBeroepsbevolking_1,Beroepsbevolking_2,WerkzameBeroepsbevolking_3,Werknemer_4,WerknemerMetVasteArbeidsrelatie_5,...,k_12Tot20UurPerWeek_15,k_20Tot35UurPerWeek_16,k_20Tot28UurPerWeek_17,k_28Tot35UurPerWeek_18,Voltijd_19,WerklozeBeroepsbevolking_20,Werkloosheidspercentage_21,NietBeroepsbevolking_22,BrutoArbeidsparticipatie_23,NettoArbeidsparticipatie_24
0,0,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 1e kwartaal,12617,9162,8427,7205,4782,...,592,2311,1224,1087,4465,735,8.0,3454,72.6,66.8
1,1,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 2e kwartaal,12631,9182,8436,7215,4753,...,599,2321,1210,1111,4464,747,8.1,3448,72.7,66.8
2,2,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 3e kwartaal,12645,9208,8443,7204,4706,...,592,2331,1215,1117,4491,764,8.3,3437,72.8,66.8
3,3,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2013 4e kwartaal,12659,9195,8426,7171,4651,...,587,2315,1184,1131,4444,768,8.4,3464,72.6,66.6
5,5,Totaal mannen en vrouwen,15 tot 75 jaar,Totaal,2014 1e kwartaal,12665,9138,8320,7073,4613,...,556,2291,1182,1109,4392,817,8.9,3528,72.1,65.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16209,16209,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021 2e kwartaal,1,0,0,0,0,...,0,0,0,0,0,0,NaN,1,NaN,NaN
16210,16210,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021 3e kwartaal,1,0,0,0,0,...,0,0,0,0,0,0,NaN,1,NaN,NaN
16211,16211,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2021 4e kwartaal,2,0,0,0,0,...,0,0,0,0,0,0,NaN,2,NaN,NaN
16213,16213,Vrouwen,70 tot 75 jaar,Weet niet of onbekend,2022 1e kwartaal,4,0,0,0,0,...,0,0,0,0,0,0,NaN,4,NaN,NaN


In [27]:
ad_eduq = adnc_q[['Perioden', 'HoogstBehaaldOnderwijsniveau','WerkzameBeroepsbevolking_3']]

In [28]:
ad_eduq

,Perioden,HoogstBehaaldOnderwijsniveau,WerkzameBeroepsbevolking_3
0,2013 1e kwartaal,Totaal,8427
1,2013 2e kwartaal,Totaal,8436
2,2013 3e kwartaal,Totaal,8443
3,2013 4e kwartaal,Totaal,8426
5,2014 1e kwartaal,Totaal,8320
...,...,...,...
16209,2021 2e kwartaal,Weet niet of onbekend,0
16210,2021 3e kwartaal,Weet niet of onbekend,0
16211,2021 4e kwartaal,Weet niet of onbekend,0
16213,2022 1e kwartaal,Weet niet of onbekend,0


In [29]:
ad_edu_pivot = ad_eduq.pivot_table(index = "Perioden", columns="HoogstBehaaldOnderwijsniveau", values="WerkzameBeroepsbevolking_3", aggfunc='sum')



In [30]:
ad_edu_pivot

HoogstBehaaldOnderwijsniveau,1 Laag onderwijsniveau,2 Middelbaar onderwijsniveau,3 Hoog onderwijsniveau,Totaal,Weet niet of onbekend
Perioden,,,,,
2013 1e kwartaal,19476,37303,29524,87791,1497
2013 2e kwartaal,19074,37411,29720,87862,1657
2013 3e kwartaal,18955,37487,29858,87915,1613
2013 4e kwartaal,18818,37223,30094,87799,1658
2014 1e kwartaal,18694,36643,30208,86760,1217
...,...,...,...,...,...
2021 2e kwartaal,18252,37580,39486,95895,570
2021 3e kwartaal,18205,37841,40253,96877,573
2021 4e kwartaal,18215,37637,41145,97544,545


In [31]:
ad_edu_ratio = ad_edu_pivot.copy()
ad_edu_ratio['LO_ratio'] = ad_edu_ratio['1 Laag onderwijsniveau'] / ad_edu_ratio['Totaal']
ad_edu_ratio['MO_ratio'] = ad_edu_ratio['2 Middelbaar onderwijsniveau'] / ad_edu_ratio['Totaal']
ad_edu_ratio['HO_ratio'] = ad_edu_ratio['3 Hoog onderwijsniveau'] / ad_edu_ratio['Totaal']
ad_edu_ratio['Weetniet_ratio'] = ad_edu_ratio['Weet niet of onbekend'] / ad_edu_ratio['Totaal']


In [32]:
ad_edu_ratio

HoogstBehaaldOnderwijsniveau,1 Laag onderwijsniveau,2 Middelbaar onderwijsniveau,3 Hoog onderwijsniveau,Totaal,Weet niet of onbekend,LO_ratio,MO_ratio,HO_ratio,Weetniet_ratio
Perioden,,,,,,,,,
2013 1e kwartaal,19476,37303,29524,87791,1497,0.221845,0.424907,0.336299,0.017052
2013 2e kwartaal,19074,37411,29720,87862,1657,0.217090,0.425793,0.338258,0.018859
2013 3e kwartaal,18955,37487,29858,87915,1613,0.215606,0.426401,0.339623,0.018347
2013 4e kwartaal,18818,37223,30094,87799,1658,0.214330,0.423957,0.342760,0.018884
2014 1e kwartaal,18694,36643,30208,86760,1217,0.215468,0.422349,0.348179,0.014027
...,...,...,...,...,...,...,...,...,...
2021 2e kwartaal,18252,37580,39486,95895,570,0.190333,0.391887,0.411763,0.005944
2021 3e kwartaal,18205,37841,40253,96877,573,0.187919,0.390609,0.415506,0.005915
2021 4e kwartaal,18215,37637,41145,97544,545,0.186736,0.385846,0.421810,0.005587


In [33]:
ad_eduratio = ad_edu_ratio[["LO_ratio", "MO_ratio", "HO_ratio", "Weetniet_ratio"]]

In [34]:
ad_eduratio

HoogstBehaaldOnderwijsniveau,LO_ratio,MO_ratio,HO_ratio,Weetniet_ratio
Perioden,,,,
2013 1e kwartaal,0.221845,0.424907,0.336299,0.017052
2013 2e kwartaal,0.217090,0.425793,0.338258,0.018859
2013 3e kwartaal,0.215606,0.426401,0.339623,0.018347
2013 4e kwartaal,0.214330,0.423957,0.342760,0.018884
2014 1e kwartaal,0.215468,0.422349,0.348179,0.014027
...,...,...,...,...
2021 2e kwartaal,0.190333,0.391887,0.411763,0.005944
2021 3e kwartaal,0.187919,0.390609,0.415506,0.005915
2021 4e kwartaal,0.186736,0.385846,0.421810,0.005587


In [35]:
df_nl_edu = pd.merge(df_nl_q, ad_eduratio, how="left", on="Perioden")

In [36]:
df_nl_edu

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1,LO_ratio,MO_ratio,HO_ratio,Weetniet_ratio
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5,NaN,NaN,NaN,NaN
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6,NaN,NaN,NaN,NaN
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0,NaN,NaN,NaN,NaN
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7,NaN,NaN,NaN,NaN
4,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
101,126,A-U Alle economische activiteiten,2021 2e kwartaal,4.7,0.190333,0.391887,0.411763,0.005944
102,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6,0.187919,0.390609,0.415506,0.005915
103,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4,0.186736,0.385846,0.421810,0.005587
104,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3,0.187716,0.382573,0.423909,0.005791


In [37]:
#For branche division -- ratio of number of jobs per branche over all jobs
branche = pd.DataFrame(cbsodata.get_data('83451NED'))

In [38]:
branche.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132733 entries, 0 to 132732
Data columns (total 10 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   ID                                      132733 non-null  int64 
 1   KenmerkenBaan                           132733 non-null  object
 2   BedrijfstakkenBranchesSBI2008           132733 non-null  object
 3   Perioden                                132733 non-null  object
 4   BanenMetSeizoenscorrectie_1             132733 non-null  int64 
 5   BanenZonderSeizoenscorrectie_2          132733 non-null  int64 
 6   ArbeidsvolumeMetSeizoenscorrectie_3     132733 non-null  int64 
 7   ArbeidsvolumeZonderSeizoenscorrectie_4  132733 non-null  int64 
 8   MaandloonInclusiefOverwerk_5            132733 non-null  int64 
 9   MaandloonExclusiefOverwerk_6            132733 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 10.1+ MB


In [39]:
#branche
branche = branche.loc[(branche['KenmerkenBaan'].str.contains('Totaal')),:]

In [40]:
brq = branche[['ID','Perioden', 'BedrijfstakkenBranchesSBI2008','BanenMetSeizoenscorrectie_1']].copy()
conditions = [
    (branche['Perioden'].str.contains('januari') | branche['Perioden'].str.contains('februari') | branche['Perioden'].str.contains('maart')),
    (branche['Perioden'].str.contains('april') | branche['Perioden'].str.contains('mei') | branche['Perioden'].str.contains('juni')),
    (branche['Perioden'].str.contains('juli') | branche['Perioden'].str.contains('augustus') | branche['Perioden'].str.contains('september')),
    (branche['Perioden'].str.contains('oktober') | branche['Perioden'].str.contains('november') | branche['Perioden'].str.contains('december')) ]

kwartaal = ['1e kwartaal', '2e kwartaal', '3e kwartaal', '4e kwartaal']

brq['Kwartaal'] = np.select(conditions, kwartaal)

# change column Perioden into string

brq['Perioden str'] = brq['Perioden'].astype(str)

brq['Jaar'] = brq['Perioden str'].str.slice(0,4,1)

brq['Perioden'] = brq['Jaar'] + " "+ brq['Kwartaal']

In [41]:
brq

,ID,Perioden,BedrijfstakkenBranchesSBI2008,BanenMetSeizoenscorrectie_1,Kwartaal,Perioden str,Jaar
0,0,2006 1e kwartaal,A-U alle economische activiteiten,7508,1e kwartaal,2006 januari,2006
1,1,2006 1e kwartaal,A-U alle economische activiteiten,7475,1e kwartaal,2006 februari,2006
2,2,2006 1e kwartaal,A-U alle economische activiteiten,7484,1e kwartaal,2006 maart,2006
3,3,2006 2e kwartaal,A-U alle economische activiteiten,7482,2e kwartaal,2006 april,2006
4,4,2006 2e kwartaal,A-U alle economische activiteiten,7481,2e kwartaal,2006 mei,2006
...,...,...,...,...,...,...,...
5766,5766,2022 1e kwartaal,S Overige dienstverlening,130,1e kwartaal,2022 maart,2022
5767,5767,2022 2e kwartaal,S Overige dienstverlening,129,2e kwartaal,2022 april,2022
5768,5768,2022 2e kwartaal,S Overige dienstverlening,130,2e kwartaal,2022 mei,2022
5769,5769,2022 2e kwartaal,S Overige dienstverlening,129,2e kwartaal,2022 juni,2022


In [42]:
brq_group_jobs = brq.groupby(['Perioden','BedrijfstakkenBranchesSBI2008'])['BanenMetSeizoenscorrectie_1'].sum().reset_index()

In [43]:
brq_group_jobs

,Perioden,BedrijfstakkenBranchesSBI2008,BanenMetSeizoenscorrectie_1
0,2006 1e kwartaal,"A Landbouw, bosbouw en visserij",338
1,2006 1e kwartaal,A-F Landbouw en nijverheid,4089
2,2006 1e kwartaal,A-U alle economische activiteiten,22467
3,2006 1e kwartaal,B Delfstoffenwinning,21
4,2006 1e kwartaal,B-E Nijverheid (geen bouw) en energie,2591
...,...,...,...
1938,2022 3e kwartaal,P Onderwijs,590
1939,2022 3e kwartaal,Q Gezondheids- en welzijnszorg,1493
1940,2022 3e kwartaal,"R Cultuur, sport en recreatie",149
1941,2022 3e kwartaal,"R-U Cultuur, recreatie, overige diensten",277


In [44]:
br_jobs_pivot = brq_group_jobs.pivot_table(index = "Perioden", columns="BedrijfstakkenBranchesSBI2008", values="BanenMetSeizoenscorrectie_1", aggfunc='sum')


In [45]:
br_jobs_pivot

BedrijfstakkenBranchesSBI2008,"A Landbouw, bosbouw en visserij",A-F Landbouw en nijverheid,A-U alle economische activiteiten,B Delfstoffenwinning,B-E Nijverheid (geen bouw) en energie,B-F Nijverheid en energie,C Industrie,D Energievoorziening,E Waterbedrijven en afvalbeheer,F Bouwnijverheid,...,M-N Zakelijke dienstverlening,N Verhuur en overige zakelijke diensten,O Openbaar bestuur en overheidsdiensten,O-Q Overheid en zorg,O-U Niet-commerciële dienstverlening,P Onderwijs,Q Gezondheids- en welzijnszorg,"R Cultuur, sport en recreatie","R-U Cultuur, recreatie, overige diensten",S Overige dienstverlening
Perioden,,,,,,,,,,,,,,,,,,,,,
2006 1e kwartaal,338,4089,22467,21,2591,3743,2411,57,105,1149,...,3832,2440,1447,6308,7130,1459,3414,368,826,451
2006 2e kwartaal,332,4060,22454,21,2580,3728,2399,57,102,1147,...,3863,2465,1443,6312,7126,1461,3412,363,814,448
2006 3e kwartaal,341,4069,22618,21,2581,3729,2397,57,103,1149,...,3921,2518,1447,6372,7179,1467,3446,365,807,440
2006 4e kwartaal,347,4073,22764,21,2582,3733,2397,57,105,1153,...,4007,2602,1444,6378,7185,1475,3454,367,802,445
2007 1e kwartaal,315,4077,22958,21,2592,3752,2406,60,105,1159,...,4098,2652,1478,6443,7237,1492,3487,364,796,426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021 3e kwartaal,340,3900,25859,24,2529,3561,2300,96,111,1032,...,4697,3073,1674,7797,8591,1695,4425,410,801,390
2021 4e kwartaal,343,3928,26103,22,2544,3584,2313,96,111,1041,...,4748,3098,1683,7854,8657,1720,4450,411,811,390
2022 1e kwartaal,341,3939,26532,22,2552,3599,2319,97,113,1047,...,4933,3249,1695,7909,8724,1741,4473,425,812,385


In [46]:
br_jobs_pivot.columns

Index(['A Landbouw, bosbouw en visserij', 'A-F Landbouw en nijverheid',
       'A-U alle economische activiteiten', 'B Delfstoffenwinning',
       'B-E Nijverheid (geen bouw) en energie', 'B-F Nijverheid en energie',
       'C Industrie', 'D Energievoorziening',
       'E Waterbedrijven en afvalbeheer', 'F Bouwnijverheid', 'G Handel',
       'G-I Handel, vervoer en horeca', 'G-N Commerciële dienstverlening',
       'H Vervoer en opslag', 'I Horeca', 'J Informatie en communicatie',
       'K Financiële dienstverlening',
       'L Verhuur en handel van onroerend goed',
       'M Specialistische zakelijke diensten', 'M-N Zakelijke dienstverlening',
       'N Verhuur en overige zakelijke diensten',
       'O Openbaar bestuur en overheidsdiensten', 'O-Q Overheid en zorg',
       'O-U Niet-commerciële dienstverlening', 'P Onderwijs',
       'Q Gezondheids- en welzijnszorg', 'R Cultuur, sport en recreatie',
       'R-U Cultuur, recreatie, overige diensten',
       'S Overige dienstverlening']

In [47]:
br_jobs = br_jobs_pivot[['A-U alle economische activiteiten', 'Q Gezondheids- en welzijnszorg','G Handel', 'N Verhuur en overige zakelijke diensten', 'C Industrie', 'P Onderwijs', 'O Openbaar bestuur en overheidsdiensten', 'M Specialistische zakelijke diensten', 'F Bouwnijverheid' ]]


In [48]:
br_jobs_ratio = br_jobs.copy()
br_jobs_ratio['Jobs_healthcare_ratio'] = br_jobs_ratio['Q Gezondheids- en welzijnszorg'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_trade_ratio'] = br_jobs_ratio['G Handel'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_rental_ratio'] = br_jobs_ratio['N Verhuur en overige zakelijke diensten'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_industry_ratio'] = br_jobs_ratio['C Industrie'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_edu_ratio'] = br_jobs_ratio['P Onderwijs'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_gov_ratio'] = br_jobs_ratio['O Openbaar bestuur en overheidsdiensten'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_specialbusiness_ratio'] = br_jobs_ratio['M Specialistische zakelijke diensten'] / br_jobs_ratio['A-U alle economische activiteiten']
br_jobs_ratio['Jobs_construction_ratio'] = br_jobs_ratio['F Bouwnijverheid'] / br_jobs_ratio['A-U alle economische activiteiten']

In [49]:
br_jobs_ratio = br_jobs_ratio[['Jobs_healthcare_ratio','Jobs_trade_ratio','Jobs_rental_ratio', 'Jobs_industry_ratio', 'Jobs_edu_ratio', 'Jobs_gov_ratio', 'Jobs_specialbusiness_ratio', 'Jobs_construction_ratio']]

In [50]:
df_nl_edu_jobs = pd.merge(df_nl_edu, br_jobs_ratio, how="left", on="Perioden")

In [51]:
df_nl_edu_jobs

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1,LO_ratio,MO_ratio,HO_ratio,Weetniet_ratio,Jobs_healthcare_ratio,Jobs_trade_ratio,Jobs_rental_ratio,Jobs_industry_ratio,Jobs_edu_ratio,Jobs_gov_ratio,Jobs_specialbusiness_ratio,Jobs_construction_ratio
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,126,A-U Alle economische activiteiten,2021 2e kwartaal,4.7,0.190333,0.391887,0.411763,0.005944,0.173179,0.168442,0.118324,0.089859,0.065740,0.065153,0.063195,0.040251
102,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6,0.187919,0.390609,0.415506,0.005915,0.171120,0.166944,0.118837,0.088944,0.065548,0.064736,0.063266,0.039909
103,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4,0.186736,0.385846,0.421810,0.005587,0.170478,0.165613,0.118684,0.088611,0.065893,0.064475,0.063594,0.039880
104,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3,0.187716,0.382573,0.423909,0.005791,0.168589,0.165008,0.122456,0.087404,0.065619,0.063885,0.063244,0.039462


In [52]:
company_size = pd.DataFrame(cbsodata.get_data('81588NED'))

In [53]:
company_size.columns

Index(['ID', 'BedrijfstakkenBranchesSBI2008', 'Perioden', 'TotaalBedrijven_1',
       'k_1WerkzaamPersoon_2', 'k_2WerkzamePersonen_3',
       'k_3Tot5WerkzamePersonen_4', 'k_5Tot10WerkzamePersonen_5',
       'k_10Tot20WerkzamePersonen_6', 'k_20Tot50WerkzamePersonen_7',
       'k_50Tot100WerkzamePersonen_8', 'k_100Tot150WerkzamePersonen_9',
       'k_150Tot200WerkzamePersonen_10', 'k_200Tot250WerkzamePersonen_11',
       'k_250Tot500WerkzamePersonen_12', 'k_500Tot1000WerkzamePersonen_13',
       'k_1000Tot2000WerkzamePersonen_14', 'k_2000WerkzamePersonenOfMeer_15',
       'k_0Tot50WerkzamePersonen_16', 'k_0Tot250WerkzamePersonen_17',
       'k_2Tot10WerkzamePersonen_18', 'k_10Tot50WerkzamePersonen_19',
       'k_50Tot250WerkzamePersonen_20', 'TotaalNatuurlijkePersonen_21',
       'Eenmanszaak_22', 'MaatschapSamenwerking_23',
       'VennootschapOnderFirmaVOF_24', 'CommanditaireVennootschapCV_25',
       'TotaalRechtspersonen_26', 'BeslotenVennootschapBV_27',
       'NaamlozeVennootschap

In [54]:
comp_size = company_size.copy()
comp_size = comp_size.loc[(company_size['BedrijfstakkenBranchesSBI2008'] == "A-U Alle economische activiteiten"),:]
comp_size['Size_under_100'] = comp_size['k_1WerkzaamPersoon_2']+ comp_size['k_2WerkzamePersonen_3'] + comp_size['k_3Tot5WerkzamePersonen_4'] + comp_size['k_5Tot10WerkzamePersonen_5'] + comp_size['k_10Tot20WerkzamePersonen_6'] + comp_size['k_20Tot50WerkzamePersonen_7'] + comp_size['k_50Tot100WerkzamePersonen_8']
comp_size['Size_100tot500'] = comp_size['k_100Tot150WerkzamePersonen_9'] + comp_size['k_150Tot200WerkzamePersonen_10'] + comp_size['k_200Tot250WerkzamePersonen_11'] + comp_size['k_250Tot500WerkzamePersonen_12']
comp_size['Size_above_500'] = comp_size['k_500Tot1000WerkzamePersonen_13'] + comp_size['k_1000Tot2000WerkzamePersonen_14'] + comp_size['k_2000WerkzamePersonenOfMeer_15']

In [55]:
comp_size = comp_size.loc[:, ("Perioden","TotaalBedrijven_1","Size_under_100","Size_100tot500", "Size_above_500")]

In [56]:
company_ratio = comp_size.copy()
company_ratio['SizeUnder100_ratio'] = company_ratio['Size_under_100'] / company_ratio['TotaalBedrijven_1']
company_ratio['Size100to500_ratio'] = company_ratio['Size_100tot500'] / company_ratio['TotaalBedrijven_1']
company_ratio['SizeAbove500_ratio'] = company_ratio['Size_above_500'] / company_ratio['TotaalBedrijven_1']


In [57]:
company_ratio = company_ratio.loc[:, ("Perioden","SizeUnder100_ratio","Size100to500_ratio","SizeAbove500_ratio")]

In [58]:
df_nl_edu_jobs_comp = pd.merge(df_nl_edu_jobs, company_ratio, how="left", on="Perioden")

In [59]:
df_nl_edu_jobs_comp

,ID,BedrijfskenmerkenSBI2008,Perioden,Ziekteverzuimpercentage_1,LO_ratio,MO_ratio,HO_ratio,Weetniet_ratio,Jobs_healthcare_ratio,Jobs_trade_ratio,Jobs_rental_ratio,Jobs_industry_ratio,Jobs_edu_ratio,Jobs_gov_ratio,Jobs_specialbusiness_ratio,Jobs_construction_ratio,SizeUnder100_ratio,Size100to500_ratio,SizeAbove500_ratio
0,0,A-U Alle economische activiteiten,1996 1e kwartaal,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A-U Alle economische activiteiten,1996 2e kwartaal,4.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A-U Alle economische activiteiten,1996 3e kwartaal,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A-U Alle economische activiteiten,1996 4e kwartaal,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,A-U Alle economische activiteiten,1997 1e kwartaal,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,126,A-U Alle economische activiteiten,2021 2e kwartaal,4.7,0.190333,0.391887,0.411763,0.005944,0.173179,0.168442,0.118324,0.089859,0.065740,0.065153,0.063195,0.040251,0.995932,0.003247,0.000821
102,127,A-U Alle economische activiteiten,2021 3e kwartaal,4.6,0.187919,0.390609,0.415506,0.005915,0.171120,0.166944,0.118837,0.088944,0.065548,0.064736,0.063266,0.039909,0.996035,0.003171,0.000792
103,128,A-U Alle economische activiteiten,2021 4e kwartaal,5.4,0.186736,0.385846,0.421810,0.005587,0.170478,0.165613,0.118684,0.088611,0.065893,0.064475,0.063594,0.039880,0.996056,0.003143,0.000799
104,130,A-U Alle economische activiteiten,2022 1e kwartaal,6.3,0.187716,0.382573,0.423909,0.005791,0.168589,0.165008,0.122456,0.087404,0.065619,0.063885,0.063244,0.039462,0.996101,0.003109,0.000792


In [60]:
occupational_health = pd.read_csv('beroepsziekten_pbi.csv', sep = ',', header=0)

In [61]:
occupational_health

,Jaar,Diagnose,Aantal,PCT
0,2003,adhesieve capsulitis van schouder,67,1.1
1,2003,"artrose van knie, gonartrose",46,0.8
2,2003,burn out,295,4.9
3,2003,carpaal-tunnelsyndroom,79,1.3
4,2003,cervicobrachiaal syndroom,31,0.5
...,...,...,...,...
237,2021,ziekten van oog en adnexen,3,0.1
238,2021,ziekten van oor en processus mastoideus,103,2.3
239,2021,ziekten van spijsverteringsstelsel,9,0.2
240,2021,ziekten van urogenitaal stelsel,2,0.0


In [62]:
occupy_health_pct = occupational_health[['Jaar', 'Diagnose', 'PCT']]

In [63]:
occupyhealthpct_pivot = occupy_health_pct.pivot_table(index = "Jaar", columns="Diagnose", values="PCT", aggfunc='sum')


In [64]:
occupyhealthpct_pivot

Diagnose,aandoeningen niet elders geclassificeerd,"aandoeningen, niet elders geclassificeerd",adhesieve capsulitis van schouder,"artrose van knie, gonartrose",burn out,carpaal-tunnelsyndroom,carpale-tunnelsyndroom,cervicobrachiaal syndroom,"combinatie rsi, geen regio","contactdermatitis, contacteczeem",...,ziekten van botspierstelsel en bindweefsel,ziekten van hartvaatstelsel,"ziekten van huid, subcutis en adnexen",ziekten van oog en adnexen,ziekten van oor en processus mastoideus,ziekten van oor en processus mastoïdeus,ziekten van spijsverteringsstelsel,ziekten van urogenitaal stelsel,ziekten van zenuwstelsel,"zwangerschap, bevalling en kraambed"
Jaar,,,,,,,,,,,,,,,,,,,,,
2003,NaN,NaN,1.1,0.8,4.9,1.3,NaN,0.5,0.8,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,NaN,NaN,0.8,0.9,6.0,NaN,1.0,0.7,0.8,2.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,3.0,0.0,28.0,NaN,1.0,0.0,2.0,NaN
2007,NaN,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.7,3.0,0.2,NaN,31.1,0.5,0.0,1.7,NaN
2008,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.8,2.6,0.3,33.8,NaN,0.8,0.0,1.4,NaN
2009,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.5,1.9,0.1,NaN,46.9,0.8,0.0,1.1,NaN
2010,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.3,0.7,2.5,0.1,41.6,NaN,0.5,0.0,1.1,NaN
2011,NaN,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.7,0.5,2.7,0.2,39.5,NaN,0.4,NaN,1.3,NaN
2012,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.8,0.5,2.2,0.2,43.7,NaN,0.2,0.0,1.4,NaN


In [65]:
df = pd.read_csv("occupyhealth_top4.csv", sep = ';', decimal=",", low_memory=False)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Perioden                                   19 non-null     int64  
 1   1_ziektenbotspierstelsel                   19 non-null     float64
 2   v1                                         19 non-null     float64
 3   2_psychische klachten en surmenage         19 non-null     float64
 4   v2                                         19 non-null     float64
 5   3_ziekten van ademhalingsstelsel           19 non-null     float64
 6   v3                                         19 non-null     float64
 7   4_ziekten van oor en processus mastoideus  19 non-null     float64
 8   v4                                         19 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 1.5 KB


In [67]:
#df #= df[['Perioden', '1_ziektenbotspierstelsel', 'v1']]
df['1_Q1'] = df['1_ziektenbotspierstelsel']
df['1_Q2'] = df['1_ziektenbotspierstelsel'] + df['v1']
df['1_Q3'] = df['1_ziektenbotspierstelsel'] + 2*df['v1']
df['1_Q4'] = df['1_ziektenbotspierstelsel'] + 3*df['v1']

df['2_Q1'] = df['2_psychische klachten en surmenage']
df['2_Q2'] = df['2_psychische klachten en surmenage'] + df['v2']
df['2_Q3'] = df['2_psychische klachten en surmenage'] + 2*df['v2']
df['2_Q4'] = df['2_psychische klachten en surmenage'] + 3*df['v2']

df['3_Q1'] = df['3_ziekten van ademhalingsstelsel']
df['3_Q2'] = df['3_ziekten van ademhalingsstelsel'] + df['v3']
df['3_Q3'] = df['3_ziekten van ademhalingsstelsel'] + 2*df['v3']
df['3_Q4'] = df['3_ziekten van ademhalingsstelsel'] + 3*df['v3']

df['4_Q1'] = df['4_ziekten van oor en processus mastoideus']
df['4_Q2'] = df['4_ziekten van oor en processus mastoideus'] + df['v4']
df['4_Q3'] = df['4_ziekten van oor en processus mastoideus'] + 2*df['v4']
df['4_Q4'] = df['4_ziekten van oor en processus mastoideus'] + 3*df['v4']

df['StartPeriode'] = df['Perioden'].astype(str)
df['EindPeriode'] = df['Perioden'] + 1
df['EindPeriode'] = df['EindPeriode'].astype(str)

In [68]:
df1 = df.copy()
df1['StartPeriode'] = df1['StartPeriode']+ '0101'
df1['StartPeriode'] = pd.to_datetime(df1['StartPeriode'], format='%Y%m%d')
df1['EindPeriode'] = df1['EindPeriode']+ '0101'
df1['EindPeriode'] = pd.to_datetime(df1['EindPeriode'], format='%Y%m%d')

In [69]:
df1
df1 = df1[df1['Perioden'] < 2022]

In [70]:
#explode year to quarter period
# df2 = df1.set_index("Perioden").apply(
#         lambda row: pd.date_range(row["StartPeriode"], row["EindPeriode"], freq="Q").values, axis=1
#         ).explode()

In [71]:
# #merge year set with quarter period
# df3 = pd.merge(df1, df2.to_frame(), how="left", on= "Perioden")

In [72]:
# df3

In [73]:
df_botspier = df1.copy()
df_botspier = df_botspier[['Perioden', '1_Q1', '1_Q2', '1_Q3', '1_Q4']]
df_botspier = df_botspier.rename(columns={'1_Q1' : '1e kwartaal', 
                '1_Q2': '2e kwartaal', 
                '1_Q3': '3e kwartaal', 
                '1_Q4': '4e kwartaal'})
df_botspier = (df_botspier.melt(id_vars='Perioden', var_name='Kwartaal',value_name='ziektenbotspierstelsel_PCT')
       .sort_values('Perioden')
       #.reset_index(drop=True)
       )


In [74]:
df_botspier['Period'] = df_botspier['Perioden'].astype(str) + ' ' + df_botspier['Kwartaal'].astype(str)
df_botspier = df_botspier.sort_values('Period').reset_index()
df_botspier

,index,Perioden,Kwartaal,ziektenbotspierstelsel_PCT,Period
0,0,2003,1e kwartaal,26.900,2003 1e kwartaal
1,19,2003,2e kwartaal,25.925,2003 2e kwartaal
2,38,2003,3e kwartaal,24.950,2003 3e kwartaal
3,57,2003,4e kwartaal,23.975,2003 4e kwartaal
4,1,2004,1e kwartaal,23.000,2004 1e kwartaal
...,...,...,...,...,...
71,74,2020,4e kwartaal,15.325,2020 4e kwartaal
72,18,2021,1e kwartaal,14.900,2021 1e kwartaal
73,37,2021,2e kwartaal,11.175,2021 2e kwartaal
74,56,2021,3e kwartaal,7.450,2021 3e kwartaal


In [75]:
df_psychisch = df1.copy()
df_psychisch = df_psychisch[['Perioden', '2_Q1', '2_Q2', '2_Q3', '2_Q4']]
df_psychisch = df_psychisch.rename(columns={'2_Q1' : '1e kwartaal', 
                '2_Q2': '2e kwartaal', 
                '2_Q3': '3e kwartaal', 
                '2_Q4': '4e kwartaal'})
df_psychisch = (df_psychisch.melt(id_vars='Perioden', var_name='Kwartaal',value_name='psychisch_PCT')
       .sort_values('Perioden')
       .reset_index(drop=True))

In [76]:
df_psychisch['Period'] = df_psychisch['Perioden'].astype(str) + ' ' + df_psychisch['Kwartaal'].astype(str)
df_psychisch = df_psychisch.sort_values('Period').reset_index()

In [77]:
df_ademhaling = df1.copy()
df_ademhaling = df_ademhaling[['Perioden', '3_Q1', '3_Q2', '3_Q3', '3_Q4']]
df_ademhaling = df_ademhaling.rename(columns={'3_Q1' : '1e kwartaal', 
                '3_Q2': '2e kwartaal', 
                '3_Q3': '3e kwartaal', 
                '3_Q4': '4e kwartaal'})
df_ademhaling = (df_ademhaling.melt(id_vars='Perioden', var_name='Kwartaal',value_name='ademhaling_PCT')
       .sort_values('Perioden')
       .reset_index(drop=True))

In [78]:
df_ademhaling['Period'] = df_ademhaling['Perioden'].astype(str) + ' ' + df_ademhaling['Kwartaal'].astype(str)
df_ademhaling = df_ademhaling.sort_values('Period').reset_index()

In [79]:
df_oorziekte = df1.copy()
df_oorziekte = df_oorziekte[['Perioden', '4_Q1', '4_Q2', '4_Q3', '4_Q4']]
df_oorziekte = df_oorziekte.rename(columns={'4_Q1' : '1e kwartaal', 
                '4_Q2': '2e kwartaal', 
                '4_Q3': '3e kwartaal', 
                '4_Q4': '4e kwartaal'})
df_oorziekte = (df_oorziekte.melt(id_vars='Perioden', var_name='Kwartaal',value_name='oorziekte_PCT')
       .sort_values('Perioden')
       .reset_index(drop=True))

In [80]:
df_oorziekte['Period'] = df_oorziekte['Perioden'].astype(str) + ' ' + df_oorziekte['Kwartaal'].astype(str)
df_oorziekte = df_oorziekte.sort_values('Period').reset_index()

In [81]:
df_ademhaling = df_ademhaling[['Period', 'ademhaling_PCT']]
df_oorziekte = df_oorziekte[['Period', 'oorziekte_PCT']]
df_botspier = df_botspier[['Period', 'ziektenbotspierstelsel_PCT']]

In [82]:
df_psychisch = df_psychisch[['Period', 'psychisch_PCT']]
df_psychisch

,Period,psychisch_PCT
0,2003 1e kwartaal,16.300
1,2003 2e kwartaal,16.875
2,2003 3e kwartaal,17.450
3,2003 4e kwartaal,18.025
4,2004 1e kwartaal,18.600
...,...,...
71,2020 4e kwartaal,36.975
72,2021 1e kwartaal,36.800
73,2021 2e kwartaal,27.600
74,2021 3e kwartaal,18.400


In [83]:
df_psychisch10 = df_psychisch[['Period', 'psychisch_PCT']].copy()
df_psychisch10['Psy_Prior1'] = df_psychisch10['psychisch_PCT'].shift(10)
df_psychisch10['Psy_Prior2'] = df_psychisch10['psychisch_PCT'].shift(9)
df_psychisch10['Psy_Prior3'] = df_psychisch10['psychisch_PCT'].shift(8)
df_psychisch10['Psy_Prior4'] = df_psychisch10['psychisch_PCT'].shift(7)
df_psychisch10['Psy_Prior5'] = df_psychisch10['psychisch_PCT'].shift(6)
df_psychisch10['Psy_Prior6'] = df_psychisch10['psychisch_PCT'].shift(5)
df_psychisch10['Psy_Prior7'] = df_psychisch10['psychisch_PCT'].shift(4)
df_psychisch10['Psy_Prior8'] = df_psychisch10['psychisch_PCT'].shift(3)
df_psychisch10['Psy_Prior9'] = df_psychisch10['psychisch_PCT'].shift(2)
df_psychisch10['Psy_Prior10'] = df_psychisch10['psychisch_PCT'].shift(1)

In [84]:
df_botspier10 = df_botspier[['Period', 'ziektenbotspierstelsel_PCT']].copy()
df_botspier10['Botspier_Prior1'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(10)
df_botspier10['Botspier_Prior2'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(9)
df_botspier10['Botspier_Prior3'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(8)
df_botspier10['Botspier_Prior4'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(7)
df_botspier10['Botspier_Prior5'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(6)
df_botspier10['Botspier_Prior6'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(5)
df_botspier10['Botspier_Prior7'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(4)
df_botspier10['Botspier_Prior8'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(3)
df_botspier10['Botspier_Prior9'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(2)
df_botspier10['Botspier_Prior10'] = df_botspier10['ziektenbotspierstelsel_PCT'].shift(1)

In [85]:
df_ademhaling10 = df_ademhaling[['Period', 'ademhaling_PCT']].copy()
df_ademhaling10['Adem_Prior1'] = df_ademhaling10['ademhaling_PCT'].shift(10)
df_ademhaling10['Adem_Prior2'] = df_ademhaling10['ademhaling_PCT'].shift(9)
df_ademhaling10['Adem_Prior3'] = df_ademhaling10['ademhaling_PCT'].shift(8)
df_ademhaling10['Adem_Prior4'] = df_ademhaling10['ademhaling_PCT'].shift(7)
df_ademhaling10['Adem_Prior5'] = df_ademhaling10['ademhaling_PCT'].shift(6)
df_ademhaling10['Adem_Prior6'] = df_ademhaling10['ademhaling_PCT'].shift(5)
df_ademhaling10['Adem_Prior7'] = df_ademhaling10['ademhaling_PCT'].shift(4)
df_ademhaling10['Adem_Prior8'] = df_ademhaling10['ademhaling_PCT'].shift(3)
df_ademhaling10['Adem_Prior9'] = df_ademhaling10['ademhaling_PCT'].shift(2)
df_ademhaling10['Adem_Prior10'] = df_ademhaling10['ademhaling_PCT'].shift(1)

In [86]:
df_oorziekte10 = df_oorziekte[['Period', 'oorziekte_PCT']].copy()
df_oorziekte10['Oor_Prior1'] = df_oorziekte10['oorziekte_PCT'].shift(10)
df_oorziekte10['Oor_Prior2'] = df_oorziekte10['oorziekte_PCT'].shift(9)
df_oorziekte10['Oor_Prior3'] = df_oorziekte10['oorziekte_PCT'].shift(8)
df_oorziekte10['Oor_Prior4'] = df_oorziekte10['oorziekte_PCT'].shift(7)
df_oorziekte10['Oor_Prior5'] = df_oorziekte10['oorziekte_PCT'].shift(6)
df_oorziekte10['Oor_Prior6'] = df_oorziekte10['oorziekte_PCT'].shift(5)
df_oorziekte10['Oor_Prior7'] = df_oorziekte10['oorziekte_PCT'].shift(4)
df_oorziekte10['Oor_Prior8'] = df_oorziekte10['oorziekte_PCT'].shift(3)
df_oorziekte10['Oor_Prior9'] = df_oorziekte10['oorziekte_PCT'].shift(2)
df_oorziekte10['Oor_Prior10'] = df_oorziekte10['oorziekte_PCT'].shift(1)

In [87]:
from functools import reduce
dfs = [df_psychisch10, df_botspier10, df_ademhaling10, df_oorziekte10]
df_occupy = reduce(lambda left,right: pd.merge(left,right,on='Period'), dfs)

In [88]:
df_occupy

,Period,psychisch_PCT,Psy_Prior1,Psy_Prior2,Psy_Prior3,Psy_Prior4,Psy_Prior5,Psy_Prior6,Psy_Prior7,Psy_Prior8,...,Oor_Prior1,Oor_Prior2,Oor_Prior3,Oor_Prior4,Oor_Prior5,Oor_Prior6,Oor_Prior7,Oor_Prior8,Oor_Prior9,Oor_Prior10
0,2003 1e kwartaal,16.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003 2e kwartaal,16.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.200
2,2003 3e kwartaal,17.450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.200,24.850
3,2003 4e kwartaal,18.025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.20,24.850,24.500
4,2004 1e kwartaal,18.600,NaN,NaN,NaN,NaN,NaN,NaN,16.300,16.875,...,NaN,NaN,NaN,NaN,NaN,NaN,25.200,24.85,24.500,24.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2020 4e kwartaal,36.975,58.850,59.300,59.750,60.200,54.525,48.850,43.175,37.500,...,6.600,6.000,5.400,4.800,3.925,3.050,2.175,1.30,1.550,1.800
72,2021 1e kwartaal,36.800,59.300,59.750,60.200,54.525,48.850,43.175,37.500,37.325,...,6.000,5.400,4.800,3.925,3.050,2.175,1.300,1.55,1.800,2.050
73,2021 2e kwartaal,27.600,59.750,60.200,54.525,48.850,43.175,37.500,37.325,37.150,...,5.400,4.800,3.925,3.050,2.175,1.300,1.550,1.80,2.050,2.300
74,2021 3e kwartaal,18.400,60.200,54.525,48.850,43.175,37.500,37.325,37.150,36.975,...,4.800,3.925,3.050,2.175,1.300,1.550,1.800,2.05,2.300,1.725


In [89]:
df_windows10 = df_nl_q[['Perioden', 'Ziekteverzuimpercentage_1']].copy()
df_windows10['SL_Prior1'] = df_windows10['Ziekteverzuimpercentage_1'].shift(10)
df_windows10['SL_Prior2'] = df_windows10['Ziekteverzuimpercentage_1'].shift(9)
df_windows10['SL_Prior3'] = df_windows10['Ziekteverzuimpercentage_1'].shift(8)
df_windows10['SL_Prior4'] = df_windows10['Ziekteverzuimpercentage_1'].shift(7)
df_windows10['SL_Prior5'] = df_windows10['Ziekteverzuimpercentage_1'].shift(6)
df_windows10['SL_Prior6'] = df_windows10['Ziekteverzuimpercentage_1'].shift(5)
df_windows10['SL_Prior7'] = df_windows10['Ziekteverzuimpercentage_1'].shift(4)
df_windows10['SL_Prior8'] = df_windows10['Ziekteverzuimpercentage_1'].shift(3)
df_windows10['SL_Prior9'] = df_windows10['Ziekteverzuimpercentage_1'].shift(2)
df_windows10['SL_Prior10'] = df_windows10['Ziekteverzuimpercentage_1'].shift(1)


In [90]:
health = pd.DataFrame(cbsodata.get_data('37852'))

In [91]:
health.columns

Index(['ID', 'Perioden', 'BevolkingOp1Januari_1', 'Levendgeborenen_2',
       'Overledenen_3', 'PersonenMetZeerGoedeGezondheid_4',
       'PersonenMetVolledigKunstgebit_5', 'Bof_6', 'Buiktyfus_7', 'Difterie_8',
       ...
       'ZiektenVanDeSpijsverteringsorganen_137',
       'TotaalZktnUrinewegenGeslachtsorganen_138',
       'NierontstekingAandAanNierbuisjes_139',
       'ComplicatiesZwangerschapBevallin_140', 'AangeborenAfwijkingen_141',
       'TotaalUitwendigeDoodsoorzaken_142', 'Vervoersongevallen_143',
       'AccidenteleVal_144', 'AccidenteleVerdrinking_145', 'Zelfdoding_146'],
      dtype='object', length=148)

In [92]:
yearly_sickleave = health[['Perioden','Ziekteverzuimpercentage_25']]
yearly_sickleave = yearly_sickleave.dropna().reset_index()

In [93]:
yearly_sickleave

,index,Perioden,Ziekteverzuimpercentage_25
0,52,1952,3.9
1,53,1953,4.1
2,54,1954,4.1
3,55,1955,4.7
4,56,1956,4.7
...,...,...,...
64,117,2017,4.0
65,118,2018,4.3
66,119,2019,4.4
67,120,2020,4.7


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a01c261-059a-4c14-bde8-f215bee5603a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>